In [1]:
import pandas as pd
import string
from collections import Iterable
from nltk.tag import ClassifierBasedTagger
from nltk.chunk import ChunkParserI, conlltags2tree
from nltk.stem.snowball import SnowballStemmer
from nltk import pos_tag, word_tokenize
from spacy import displacy

from newspaper import Article
import re

class EntityChunker(ChunkParserI):
    def __init__(self, train_sents, **kwargs):
        assert isinstance(train_sents, Iterable)

        self.feature_detector = features
        self.tagger = ClassifierBasedTagger(
            train=train_sents,
            feature_detector=features,
            **kwargs)

    def parse(self, tagged_sent):
        chunks = self.tagger.tag(tagged_sent)
        iob_triplets = [(w, t, c) for ((w, t), c) in chunks]
        
        return iob_triplets


def features(tokens, index, history):
    """
    'tokens'  = a POS-tagged sentence [(w1, t1), ...]
    'index'   = the index of the token we want to extract features for
    'history' = the previous predicted IOB tags
    """

    # init the stemmer
    stemmer = SnowballStemmer('english')

    # Pad the sequence with placeholders
    tokens = [('[START2]', '[START2]'), ('[START1]', '[START1]')] + list(tokens) + [('[END1]', '[END1]'),
                                                                                    ('[END2]', '[END2]')]
    history = ['[START2]', '[START1]'] + list(history)

    # shift the index with 2, to accommodate the padding
    index += 2

    word, pos = tokens[index]
    prevword, prevpos = tokens[index - 1]
    prevprevword, prevprevpos = tokens[index - 2]
    nextword, nextpos = tokens[index + 1]
    nextnextword, nextnextpos = tokens[index + 2]
    previob = history[index - 1]
    contains_dash = '-' in word
    contains_dot = '.' in word
    allascii = all([True for c in word if c in string.ascii_lowercase])

    allcaps = word == word.capitalize()
    capitalized = word[0] in string.ascii_uppercase

    prevallcaps = prevword == prevword.capitalize()
    prevcapitalized = prevword[0] in string.ascii_uppercase

    nextallcaps = prevword == prevword.capitalize()
    nextcapitalized = prevword[0] in string.ascii_uppercase

    return {
        'word': word,
        'lemma': stemmer.stem(word),
        'pos': pos,
        'all-ascii': allascii,

        'next-word': nextword,
        'next-lemma': stemmer.stem(nextword),
        'next-pos': nextpos,

        'next-next-word': nextnextword,
        'nextnextpos': nextnextpos,

        'prev-word': prevword,
        'prev-lemma': stemmer.stem(prevword),
        'prev-pos': prevpos,

        'prev-prev-word': prevprevword,
        'prev-prev-pos': prevprevpos,

        'prev-iob': previob,

        'contains-dash': contains_dash,
        'contains-dot': contains_dot,

        'all-caps': allcaps,
        'capitalized': capitalized,

        'prev-all-caps': prevallcaps,
        'prev-capitalized': prevcapitalized,

        'next-all-caps': nextallcaps,
        'next-capitalized': nextcapitalized,
    }

In [2]:
# Read data
data_file_path = '/Users/TonY/Desktop/sfl/Dataset_01-29-2019.txt'
data = pd.read_csv(data_file_path, delimiter='\t',
                   header=0, index_col=0,
                   names=['Sentence', 'Word', 'POS', 'Tag'],
                   encoding='unicode_escape')
# Check NA
data.isna().any()

# Check datatype
data.dtypes

# Change float to int
data.Sentence = data.Sentence.astype(int)
data[1:10]

,Sentence,Word,POS,Tag
1,1,of,IN,O
2,1,demonstrators,NNS,O
3,1,have,VBP,O
4,1,marched,VBN,O
5,1,through,IN,O
6,1,London,NNP,B-geo
7,1,to,TO,O
8,1,protest,VB,O
9,1,the,DT,O


In [3]:
# Total number of Sentences
max(data.Sentence)

# Tag Count by category
# ART, EVE, NAT has too few data points
data.Tag.value_counts()

O        56217
B-geo     2070
B-org     1237
I-per     1234
B-gpe     1230
B-tim     1160
B-per     1107
I-org      926
I-geo      414
I-tim      334
B-art       53
B-eve       45
I-eve       37
I-gpe       34
I-art       34
B-nat       20
I-nat        9
Name: Tag, dtype: int64

In [4]:
sentence_arr = data.Sentence.unique()
token = []

for sentence in sentence_arr:
    temp_data = data[data['Sentence'] == sentence]
    curr_index = temp_data.index
    one_sentence = []
    for index in curr_index:
        one_sentence.append(((temp_data.Word[index], temp_data.POS[index]), temp_data.Tag[index]))
    token.append(one_sentence)

training_samples = token[:int(len(token) * 0.9)]
test_samples = token[:int(len(token) * 0.1)]

chunker = EntityChunker(training_samples[:2000])

# Accuracy
score = chunker.evaluate([conlltags2tree([(w, t, iob) for (w, t), iob in iobs]) for iobs in test_samples[:500]])
print(score.accuracy())

0.8520664938233948


In [10]:

def url2text(url):
    """
    This function takes in an url of the news article and output as string of text.
    Returns a string with formated data, pads before and after each special symbol
    """
    toi_article = Article(url, language="en") # en for English
    toi_article.download() 
  
    #To parse the article 
    toi_article.parse()
    
    #To extract text 
    test_sentence = toi_article.text
    # Padding one space before and after the special symbol, not pad if already padded
    test_sentence = re.sub('(?<! )(?=[.,!?%#^&*@$:;()"\'`-])|(?<=[.,!?%#^&*@$:;()"\'`-])(?! )', r' ', test_sentence)
    
    return test_sentence


# test_sentence = "Georgia's Augusta National Golf Club is included in the slight risk area Sunday as the Masters concludes."

# # ori = ['.', ',', '(', ')', '!', '']
# # test_sentence.replace('.', ' .', test_sentence.count('.'))
# test_sentence.replace(',', ' ,', test_sentence.count(','))


# test_sentence = 'an enhanced risk(level 3 out of 5)stretches from Atlanta to Cleveland.'
# test_sentence = re.sub('(?<=\w)([''])', r' \1', test_sentence)
# test_sentence = re.sub(r"(?<!\s)\(", r' (', test_sentence) # => " (some text)"

# Padding one space before and after the special symbol, not pad if already padded
# test_sentence = re.sub('(?<! )(?=[.,!?()"\'`])|(?<=[.,!?()"\'`])(?! )', r' ', test_sentence)
# print(test_sentence)


# IOB

def render_nltk_ner(test_sentence):
    """
    This function serves to take the text output of the function "url2text", 
    restructure the data and then renders using SpaCy API. 
    It only shows the rendered text"
    """
    for line in test_sentence.splitlines():
        iob_out = chunker.parse(pos_tag(word_tokenize(line)))
#         # Print iob token
#         print(iob_out)

        ent_hash = {}
        ent_list = []
        hash_list = []
        sentence = []

        start_index = 0
        for i, iob in enumerate(iob_out):
            sentence.append(iob[0])
            sentence_full = ' '.join(map(str, sentence))
            if iob[2].startswith('B-'):
                temp = [iob_out[i][0]]
                try:
                    iob_out[i+1][2]
                    if iob_out[i+1][2] == 'O':
                        word = ' '.join(map(str, temp))
                        ent_hash = {'word': word,
                                    "start": len(sentence_full) - len(word),
                                    "end": len(sentence_full),
                                    "label": iob_out[i][2][2:].upper()}

                        hash_list.append(ent_hash)
                except IndexError:
                     break
                        
            elif iob[2].startswith('I-'):
                temp.append(iob_out[i][0])
                try:
                    iob_out[i+1][2]
                    if iob_out[i+1][2] == 'O':
                        word = ' '.join(map(str, temp))
                        ent_hash = {'word': word,
                                    "start": len(sentence_full) - len(word),
                                    "end": len(sentence_full),
                                    "label": iob_out[i][2][2:].upper()}
                        hash_list.append(ent_hash)
                except IndexError:
                     break
                    

        ent = {'text': line,
               'ents': hash_list,
               'title': None}

        displacy.render(ent, manual=True, style="ent", jupyter=True)
        
    return 


def render_nltk_ner_with_iob(test_sentence):
    """
    This function serves to take the text output of the function "url2text", 
    restructure the data and then renders using SpaCy API. 
    It shows both iob token and the rendered text"
    """
    
    for line in test_sentence.splitlines():
        iob_out = chunker.parse(pos_tag(word_tokenize(line)))
        # Print iob token
        print(iob_out)

        ent_hash = {}
        ent_list = []
        hash_list = []
        sentence = []

        start_index = 0
        for i, iob in enumerate(iob_out):
            sentence.append(iob[0])
            sentence_full = ' '.join(map(str, sentence))
            if iob[2].startswith('B-'):
                temp = [iob_out[i][0]]
                try:
                    iob_out[i+1][2]
                    if iob_out[i+1][2] == 'O':
                        word = ' '.join(map(str, temp))
                        ent_hash = {'word': word,
                                    "start": len(sentence_full) - len(word),
                                    "end": len(sentence_full),
                                    "label": iob_out[i][2][2:].upper()}

                        hash_list.append(ent_hash)
                except IndexError:
                     break
                        
            elif iob[2].startswith('I-'):
                temp.append(iob_out[i][0])
                try:
                    iob_out[i+1][2]
                    if iob_out[i+1][2] == 'O':
                        word = ' '.join(map(str, temp))
                        ent_hash = {'word': word,
                                    "start": len(sentence_full) - len(word),
                                    "end": len(sentence_full),
                                    "label": iob_out[i][2][2:].upper()}
                        hash_list.append(ent_hash)
                except IndexError:
                     break
                    

        ent = {'text': line,
               'ents': hash_list,
               'title': None}

        displacy.render(ent, manual=True, style="ent", jupyter=True)
        
    return 

In [11]:
url = "https://www.cnn.com/2019/04/13/politics/tax-reform-winners-and-losers/index.html"

text = url2text(url)
render_nltk_ner(text)


In [7]:
render_nltk_ner_with_iob(text)

[('(', '(', 'B-per'), ('CNN', 'NNP', 'B-org'), (')', ')', 'I-org'), ('Tax', 'NNP', 'B-tim'), ('day', 'NN', 'I-tim'), ('isn', 'NN', 'I-tim'), ("'", "''", 'I-tim'), ('t', 'NN', 'I-tim'), ('until', 'IN', 'I-tim'), ('Monday', 'NNP', 'B-tim'), (',', ',', 'O'), ('but', 'CC', 'O'), ('there', 'RB', 'O'), ('have', 'VBP', 'O'), ('already', 'RB', 'O'), ('been', 'VBN', 'O'), ('some', 'DT', 'O'), ('surprises', 'NNS', 'O'), ('for', 'IN', 'O'), ('Americans', 'NNPS', 'B-gpe'), ('filing', 'VBG', 'O'), ('their', 'PRP$', 'O'), ('first', 'JJ', 'O'), ('income', 'NN', 'O'), ('tax', 'NN', 'O'), ('returns', 'NNS', 'O'), ('under', 'IN', 'O'), ('President', 'NNP', 'B-per'), ('Donald', 'NNP', 'I-per'), ('Trump', 'NNP', 'I-per'), ("'", 'POS', 'I-org'), ('s', 'NN', 'I-tim'), ('2017', 'CD', 'I-tim'), ('law', 'NN', 'I-tim'), ('.', '.', 'O')]


[]


[('Let', 'VB', 'B-per'), ("'", "''", 'I-per'), ('s', 'JJ', 'O'), ('start', 'NN', 'O'), ('with', 'IN', 'O'), ('the', 'DT', 'O'), ('big', 'JJ', 'O'), ('one', 'NN', 'O'), ('.', '.', 'O'), ('While', 'IN', 'O'), ('most', 'JJS', 'O'), ('Americans', 'NNPS', 'B-gpe'), ('are', 'VBP', 'O'), ('paying', 'VBG', 'O'), ('less', 'RBR', 'O'), ('in', 'IN', 'O'), ('taxes', 'NNS', 'O'), ('overall', 'JJ', 'O'), (',', ',', 'O'), ('many', 'JJ', 'O'), ('have', 'VBP', 'O'), ('been', 'VBN', 'O'), ('startled', 'VBN', 'O'), ('to', 'TO', 'O'), ('find', 'VB', 'O'), ('that', 'DT', 'O'), ('their', 'PRP$', 'O'), ('refunds', 'NNS', 'O'), ('have', 'VBP', 'O'), ('barely', 'RB', 'O'), ('changed', 'VBN', 'O'), ('or', 'CC', 'O'), ('are', 'VBP', 'O'), ('down', 'RP', 'O'), ('-', ':', 'O'), ('-', ':', 'O'), ('making', 'VBG', 'O'), ('them', 'PRP', 'O'), ('feel', 'VBP', 'O'), ('like', 'IN', 'O'), ('they', 'PRP', 'O'), ('lost', 'VBD', 'O'), (',', ',', 'O'), ('even', 'RB', 'O'), ('if', 'IN', 'O'), ('they', 'PRP', 'O'), ("'", 'VBP'

[]


[('Below', 'NNP', 'B-gpe'), ('are', 'VBP', 'O'), ('some', 'DT', 'O'), ('other', 'JJ', 'O'), ('ways', 'NNS', 'O'), ('to', 'TO', 'O'), ('look', 'VB', 'O'), ('at', 'IN', 'O'), ('who', 'WP', 'O'), ('is', 'VBZ', 'O'), ('winning', 'VBG', 'O'), ('and', 'CC', 'O'), ('losing', 'VBG', 'O'), ('under', 'IN', 'O'), ('the', 'DT', 'O'), ('law', 'NN', 'O'), ('.', '.', 'O'), ('And', 'CC', 'O'), ('please', 'JJ', 'O'), ('take', 'VB', 'O'), ('note', 'NN', 'O'), (':', ':', 'O'), ('It', 'PRP', 'B-nat'), ("'", "''", 'I-nat'), ('s', 'RBR', 'O'), ('possible', 'JJ', 'O'), ('to', 'TO', 'O'), ('both', 'DT', 'O'), ('win', 'NNS', 'O'), ('and', 'CC', 'O'), ('lose', 'VB', 'O'), ('at', 'IN', 'O'), ('the', 'DT', 'O'), ('same', 'JJ', 'O'), ('time', 'NN', 'O'), ('.', '.', 'O')]


...

[('Winning', 'VBG', 'B-gpe'), ('forever', 'NN', 'I-gpe'), (':', ':', 'I-tim'), ('Corporations', 'NNS', 'O')]


[]


[('They', 'PRP', 'O'), ('got', 'VBD', 'O'), ('a', 'DT', 'O'), ('massive', 'JJ', 'O'), ('rate', 'NN', 'O'), ('cut', 'NN', 'O'), (',', ',', 'O'), ('from', 'IN', 'O'), ('35', 'CD', 'O'), ('%', 'NN', 'O'), ('to', 'TO', 'O'), ('21', 'CD', 'O'), ('%', 'NN', 'O'), (',', ',', 'O'), ('starting', 'VBG', 'O'), ('in', 'IN', 'O'), ('2018', 'CD', 'B-tim'), ('that', 'IN', 'I-tim'), ("'", "''", 'I-tim'), ('s', 'NNS', 'I-tim'), ('permanently', 'RB', 'I-tim'), ('enshrined', 'VBN', 'O'), ('in', 'IN', 'O'), ('the', 'DT', 'O'), ('new', 'JJ', 'O'), ('law', 'NN', 'O'), ('.', '.', 'O')]


[]


[('Losing', 'VBG', 'B-per'), (':', ':', 'I-per'), ('The', 'DT', 'O'), ('US', 'NNP', 'B-org'), ('Treasury', 'NNP', 'O')]


In [8]:
# limitation of these datasets is that they are biased towards certain genres such as 
# GEO, ORG, PER, GPE, TIM due to the unbalanced porportion, thus the NER classifiers 
# often perform poorly when trained on these dataset and then tested on a other types 
# of text genres such as ART, EVE, NAT.

# As we can see from the output of the article above, it does not recognize the money, 
# and instead it recognizes as time since our dataset does not contain training label
# for date, one potential solution besides from making a new Tag category is that 
# we can hard code to make it learn that any numeric number after $ (dollar sign) to 
# be recognized as money.